In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import (
    load_iris,
    load_wine,
    load_breast_cancer,
    load_diabetes,
    make_friedman1,
    make_regression
)
from sklearn.metrics import accuracy_score, mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(1)
np.random.seed(1)

In [4]:

iris_data = load_iris()
iris_X, iris_y = iris_data.data, iris_data.target

wine_data = load_wine()
wine_X, wine_y = wine_data.data, wine_data.target

cancer_data = load_breast_cancer()
cancer_X, cancer_y = cancer_data.data, cancer_data.target

diabetes_data = load_diabetes()
diabetes_X, diabetes_y = diabetes_data.data, diabetes_data.target

friedman_X, friedman_y = make_friedman1(n_samples=500, n_features=10, noise=0.1, random_state=1)

synth_X, synth_y = make_regression(n_samples=500, n_features=15, n_informative=10, noise=0.1, random_state=1)


In [5]:
# Split and scale iris
iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(
    iris_X, iris_y, test_size=0.2, random_state=1, stratify=iris_y
)
scaler1 = StandardScaler()
iris_X_train = scaler1.fit_transform(iris_X_train)
iris_X_test = scaler1.transform(iris_X_test)


wine_X_train, wine_X_test, wine_y_train, wine_y_test = train_test_split(
    wine_X, wine_y, test_size=0.2, random_state=1, stratify=wine_y
)
scaler2 = StandardScaler()
wine_X_train = scaler2.fit_transform(wine_X_train)
wine_X_test = scaler2.transform(wine_X_test)


cancer_X_train, cancer_X_test, cancer_y_train, cancer_y_test = train_test_split(
    cancer_X, cancer_y, test_size=0.2, random_state=1, stratify=cancer_y
)
scaler3 = StandardScaler()
cancer_X_train = scaler3.fit_transform(cancer_X_train)
cancer_X_test = scaler3.transform(cancer_X_test)


diabetes_X_train, diabetes_X_test, diabetes_y_train, diabetes_y_test = train_test_split(
    diabetes_X, diabetes_y, test_size=0.2, random_state=1
)
scaler4 = StandardScaler()
diabetes_X_train = scaler4.fit_transform(diabetes_X_train)
diabetes_X_test = scaler4.transform(diabetes_X_test)


friedman_X_train, friedman_X_test, friedman_y_train, friedman_y_test = train_test_split(
    friedman_X, friedman_y, test_size=0.2, random_state=1
)
scaler5 = StandardScaler()
friedman_X_train = scaler5.fit_transform(friedman_X_train)
friedman_X_test = scaler5.transform(friedman_X_test)


synth_X_train, synth_X_test, synth_y_train, synth_y_test = train_test_split(
    synth_X, synth_y, test_size=0.2, random_state=1
)
scaler6 = StandardScaler()
synth_X_train = scaler6.fit_transform(synth_X_train)
synth_X_test = scaler6.transform(synth_X_test)


# Scale regression targets
target_scaler_diabetes = StandardScaler()
diabetes_y_train = target_scaler_diabetes.fit_transform(diabetes_y_train.reshape(-1,1)).flatten()
diabetes_y_test = target_scaler_diabetes.transform(diabetes_y_test.reshape(-1,1)).flatten()

target_scaler_friedman = StandardScaler()
friedman_y_train = target_scaler_friedman.fit_transform(friedman_y_train.reshape(-1,1)).flatten()
friedman_y_test = target_scaler_friedman.transform(friedman_y_test.reshape(-1,1)).flatten()

target_scaler_synth = StandardScaler()
synth_y_train = target_scaler_synth.fit_transform(synth_y_train.reshape(-1,1)).flatten()
synth_y_test = target_scaler_synth.transform(synth_y_test.reshape(-1,1)).flatten()


In [6]:
def make_net(input_dim, hidden_dim, output_dim):
    layers = []
    layers.append(nn.Linear(input_dim, hidden_dim))
    layers.append(nn.ReLU())
    layers.append(nn.Linear(hidden_dim, output_dim))
    return nn.Sequential(*layers)

def train_net(net, X_data, y_data, lr=0.01, epochs=100, is_classification=True):
    X_tensor = torch.FloatTensor(X_data)
    
    if is_classification:
        y_tensor = torch.LongTensor(y_data)
        loss_fn = nn.CrossEntropyLoss()
    else:
        y_tensor = torch.FloatTensor(y_data).reshape(-1, 1)
        loss_fn = nn.MSELoss()
    
    optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=1e-4)
    
    net.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = net(X_tensor)
        loss = loss_fn(outputs, y_tensor)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
        optimizer.step()
    
    return net

def test_net(net, X_test, y_test, is_classification=True,  target_scaler=None):
    net.eval()
    X_tensor = torch.FloatTensor(X_test)
    
    with torch.no_grad():
        outputs = net(X_tensor)
        
        if is_classification:
            preds = torch.argmax(outputs, dim=1).numpy()
            acc = accuracy_score(y_test, preds)
            return acc
        else:
            preds = outputs.numpy().flatten()
            if target_scaler is not None:
                preds_unscaled = target_scaler.inverse_transform(preds.reshape(-1,1)).flatten()
                y_test_unscaled = target_scaler.inverse_transform(np.array(y_test).reshape(-1,1)).flatten()
                mse_unscaled = mean_squared_error(y_test_unscaled, preds_unscaled)
                return mse_unscaled
            else:
                mse = mean_squared_error(y_test, preds)
                return mse

In [7]:
def random_query(net, pool_X, pool_y, n_query=1):
    idxs = np.random.choice(len(pool_X), size=min(n_query, len(pool_X)), replace=False)
    return idxs

def sasla_query(net, pool_X, pool_y, beta=0.8, n_query=1):
    net.eval()
    X_tensor = torch.FloatTensor(pool_X)
    print('Sasla running...')
    sensitivity_scores = []

    for i in range(len(pool_X)):
        x = X_tensor[i:i+1].clone().detach().requires_grad_(True)
        output = net(x)

        # Jacobian
        jac = []
        for j in range(output.shape[1]):
            grad_outputs = torch.zeros_like(output)
            grad_outputs[0, j] = 1
            grad = torch.autograd.grad(
                outputs=output,
                inputs=x,
                grad_outputs=grad_outputs,
                retain_graph=True,
                create_graph=False
            )[0]
            jac.append(grad.view(-1))
        jac = torch.stack(jac)  # shape

        # output sensitivity
        output_sens_vec = torch.sum(torch.abs(jac), dim=1)

        # informativeness
        informativeness = torch.max(torch.abs(output_sens_vec))
        sensitivity_scores.append(informativeness.item())

    sensitivity_scores = np.array(sensitivity_scores)

    # threshold
    mean_sens = np.mean(sensitivity_scores)
    thresh = (1.0 - beta) * mean_sens

    candidates = np.where(sensitivity_scores > thresh)[0]
    if len(candidates) == 0:
        candidates = [np.argmax(sensitivity_scores)]

    #selected = np.random.choice(candidates, size=min(n_query, len(candidates)), replace=False)
    return candidates


def alus_query(net, pool_X, pool_y, t_param=20, n_query=1):
    net.eval()
    X_tensor = torch.FloatTensor(pool_X)
    
    with torch.no_grad():
        outputs = net(X_tensor)
        probs = torch.softmax(outputs, dim=1)
        entropy_vals = -torch.sum(probs * torch.log(probs + 1e-10), dim=1)
    
    uncertain_idxs = torch.topk(entropy_vals, min(t_param, len(entropy_vals))).indices
    
    conflict_scores = []
    for idx in uncertain_idxs:
        prob_vec = probs[idx]
        top2_vals = torch.topk(prob_vec, 2)
        
        # Get score
        with torch.no_grad():
            hidden_output = net[0](X_tensor[idx:idx+1])
            hidden_output = net[1](hidden_output)  # RELU
            final_weights = net[2].weight
            
            evidence1 = torch.sum(torch.relu(final_weights[top2_vals.indices[0]] * hidden_output))
            evidence2 = torch.sum(torch.relu(final_weights[top2_vals.indices[1]] * hidden_output))
            conflict_score = evidence1 * evidence2
        
        conflict_scores.append((idx.item(), conflict_score.item()))
    
    conflict_scores.sort(key=lambda x: x[1], reverse=True)
    selected = [conflict_scores[i][0] for i in range(min(n_query, len(conflict_scores)))]
    return np.array(selected)

In [8]:
def do_active_learning(X_train, y_train, X_test, y_test, query_func, 
                      initial_samples=10, budget=50, hidden_size=50, 
                      lr=0.01, epochs=100, is_classification=True, target_scaler=None):
    
    labeled_idxs = np.random.choice(len(X_train), size=initial_samples, replace=False)
    unlabeled_idxs = np.setdiff1d(np.arange(len(X_train)), labeled_idxs)
    
    performance_list = []
    sizes_list = []
    
    for iter_num in range(budget):
        if is_classification:
            n_classes = len(np.unique(y_train))
        else:
            n_classes = 1
        
        net = make_net(X_train.shape[1], hidden_size, n_classes)
        net = train_net(net, X_train[labeled_idxs], y_train[labeled_idxs], 
                       lr=lr, epochs=epochs, is_classification=is_classification)
        
        target_scaler = target_scaler_diabetes
        perf = test_net(net, X_test, y_test, is_classification=is_classification, target_scaler=target_scaler)
        performance_list.append(perf)
        sizes_list.append(len(labeled_idxs))
        
        if len(unlabeled_idxs) == 0:
            break
        
        selected = query_func(net, X_train[unlabeled_idxs], y_train[unlabeled_idxs])
        
        new_labeled = unlabeled_idxs[selected]
        labeled_idxs = np.concatenate([labeled_idxs, new_labeled])
        unlabeled_idxs = np.setdiff1d(unlabeled_idxs, new_labeled)
    
    return sizes_list, performance_list

In [9]:

iris_passive_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(iris_X_train, iris_y_train, iris_X_test, iris_y_test, 
                                   random_query, initial_samples=10, budget=40, 
                                   hidden_size=30, lr=0.01, epochs=100, is_classification=True)
    iris_passive_results.append((sizes, perf))

iris_sasla_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(iris_X_train, iris_y_train, iris_X_test, iris_y_test, 
                                   sasla_query, initial_samples=10, budget=40, 
                                   hidden_size=30, lr=0.01, epochs=100, is_classification=True)
    iris_sasla_results.append((sizes, perf))

iris_alus_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(iris_X_train, iris_y_train, iris_X_test, iris_y_test, 
                                   alus_query, initial_samples=10, budget=40, 
                                   hidden_size=30, lr=0.01, epochs=100, is_classification=True)
    iris_alus_results.append((sizes, perf))


Sasla running...
Sasla running...
Sasla running...
Sasla running...
Sasla running...


In [10]:

wine_passive_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(wine_X_train, wine_y_train, wine_X_test, wine_y_test, 
                                   random_query, initial_samples=10, budget=50, 
                                   hidden_size=70, lr=0.01, epochs=100, is_classification=True)
    wine_passive_results.append((sizes, perf))


wine_sasla_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(wine_X_train, wine_y_train, wine_X_test, wine_y_test, 
                                   sasla_query, initial_samples=10, budget=50, 
                                   hidden_size=70, lr=0.01, epochs=100, is_classification=True)
    wine_sasla_results.append((sizes, perf))


wine_alus_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(wine_X_train, wine_y_train, wine_X_test, wine_y_test, 
                                   alus_query, initial_samples=10, budget=50, 
                                   hidden_size=70, lr=0.01, epochs=100, is_classification=True)
    wine_alus_results.append((sizes, perf))


Sasla running...
Sasla running...
Sasla running...
Sasla running...
Sasla running...


In [11]:

cancer_passive_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(cancer_X_train, cancer_y_train, cancer_X_test, cancer_y_test, 
                                   random_query, initial_samples=20, budget=100, 
                                   hidden_size=70, lr=0.01, epochs=100, is_classification=True)
    cancer_passive_results.append((sizes, perf))


cancer_sasla_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(cancer_X_train, cancer_y_train, cancer_X_test, cancer_y_test, 
                                   sasla_query, initial_samples=20, budget=100, 
                                   hidden_size=70, lr=0.01, epochs=100, is_classification=True)
    cancer_sasla_results.append((sizes, perf))


cancer_alus_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(cancer_X_train, cancer_y_train, cancer_X_test, cancer_y_test, 
                                   alus_query, initial_samples=20, budget=100, 
                                   hidden_size=70, lr=0.01, epochs=100, is_classification=True)
    cancer_alus_results.append((sizes, perf))


Sasla running...
Sasla running...
Sasla running...
Sasla running...
Sasla running...


In [12]:

diabetes_passive_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(diabetes_X_train, diabetes_y_train, diabetes_X_test, diabetes_y_test, 
                                   random_query, initial_samples=20, budget=100, 
                                   hidden_size=50, lr=0.01, epochs=100, is_classification=False,target_scaler=target_scaler_diabetes)
    diabetes_passive_results.append((sizes, perf))

diabetes_sasla_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(diabetes_X_train, diabetes_y_train, diabetes_X_test, diabetes_y_test, 
                                   sasla_query, initial_samples=20, budget=100, 
                                   hidden_size=50, lr=0.01, epochs=100, is_classification=False,target_scaler=target_scaler_diabetes)
    diabetes_sasla_results.append((sizes, perf))


Sasla running...
Sasla running...
Sasla running...
Sasla running...
Sasla running...


In [13]:

friedman_passive_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(friedman_X_train, friedman_y_train, friedman_X_test, friedman_y_test, 
                                   random_query, initial_samples=20, budget=100, 
                                   hidden_size=70, lr=0.005, epochs=100, is_classification=False,target_scaler=target_scaler_diabetes)
    friedman_passive_results.append((sizes, perf))

friedman_sasla_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(friedman_X_train, friedman_y_train, friedman_X_test, friedman_y_test, 
                                   sasla_query, initial_samples=20, budget=100, 
                                   hidden_size=70, lr=0.005, epochs=100, is_classification=False,target_scaler=target_scaler_diabetes)
    friedman_sasla_results.append((sizes, perf))


Sasla running...
Sasla running...
Sasla running...
Sasla running...
Sasla running...


In [14]:

synth_passive_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(synth_X_train, synth_y_train, synth_X_test, synth_y_test, 
                                   random_query, initial_samples=50, budget=100, 
                                   hidden_size=50, lr=0.01, epochs=100, is_classification=False,target_scaler=target_scaler_diabetes)
    synth_passive_results.append((sizes, perf))

synth_sasla_results = []
for trial in range(5):
    np.random.seed(trial)
    torch.manual_seed(trial)
    sizes, perf = do_active_learning(synth_X_train, synth_y_train, synth_X_test, synth_y_test, 
                                   sasla_query, initial_samples=50, budget=100, 
                                   hidden_size=50, lr=0.01, epochs=100, is_classification=False,target_scaler=target_scaler_diabetes)
    synth_sasla_results.append((sizes, perf))


Sasla running...
Sasla running...
Sasla running...
Sasla running...
Sasla running...


In [15]:
print('Iris:')
passive_final = [trial[1][-1] for trial in iris_passive_results]
sasla_final = [trial[1][-1] for trial in iris_sasla_results]
alus_final = [trial[1][-1] for trial in iris_alus_results]
print(f'Passive: {np.mean(passive_final):.4f} pm {np.std(passive_final):.4f}')
print(f'SASLA: {np.mean(sasla_final):.4f} pm {np.std(sasla_final):.4f}')
print(f'ALUS: {np.mean(alus_final):.4f} pm {np.std(alus_final):.4f}')

print('Wine:')
passive_final = [trial[1][-1] for trial in wine_passive_results]
sasla_final = [trial[1][-1] for trial in wine_sasla_results]
alus_final = [trial[1][-1] for trial in wine_alus_results]
print(f'Passive: {np.mean(passive_final):.4f} pm {np.std(passive_final):.4f}')
print(f'SASLA: {np.mean(sasla_final):.4f} pm {np.std(sasla_final):.4f}')
print(f'ALUS: {np.mean(alus_final):.4f} pm {np.std(alus_final):.4f}')

print('Cancer:')
passive_final = [trial[1][-1] for trial in cancer_passive_results]
sasla_final = [trial[1][-1] for trial in cancer_sasla_results]
alus_final = [trial[1][-1] for trial in cancer_alus_results]
print(f'Passive: {np.mean(passive_final):.4f} pm {np.std(passive_final):.4f}')
print(f'SASLA: {np.mean(sasla_final):.4f} pm {np.std(sasla_final):.4f}')
print(f'ALUS: {np.mean(alus_final):.4f} pm {np.std(alus_final):.4f}')

print('Diabbetes:')
passive_final = [trial[1][-1] for trial in diabetes_passive_results]
sasla_final = [trial[1][-1] for trial in diabetes_sasla_results]
print(f'Passive: {np.mean(passive_final):.4f} pm {np.std(passive_final):.4f}')
print(f'SASLA: {np.mean(sasla_final):.4f} pm {np.std(sasla_final):.4f}')

print('Friedman:')
passive_final = [trial[1][-1] for trial in friedman_passive_results]
sasla_final = [trial[1][-1] for trial in friedman_sasla_results]
print(f'Passive: {np.mean(passive_final):.4f} pm {np.std(passive_final):.4f}')
print(f'SASLA: {np.mean(sasla_final):.4f} pm {np.std(sasla_final):.4f}')

print('Synthetic:')
passive_final = [trial[1][-1] for trial in synth_passive_results]
sasla_final = [trial[1][-1] for trial in synth_sasla_results]
print(f'Passive: {np.mean(passive_final):.4f} pm {np.std(passive_final):.4f}')
print(f'SASLA: {np.mean(sasla_final):.4f} pm {np.std(sasla_final):.4f}')

Iris:
Passive: 0.7667 pm 0.0558
SASLA: 0.7133 pm 0.0499
ALUS: 0.8267 pm 0.0573
Wine:
Passive: 0.9222 pm 0.0593
SASLA: 0.9889 pm 0.0136
ALUS: 0.9500 pm 0.0478
Cancer:
Passive: 0.9228 pm 0.0187
SASLA: 0.9333 pm 0.0070
ALUS: 0.9544 pm 0.0102
Diabbetes:
Passive: 3330.9931 pm 176.9137
SASLA: 3219.7515 pm 81.9838
Friedman:
Passive: 3479.9149 pm 249.6106
SASLA: 3211.2862 pm 289.1304
Synthetic:
Passive: 1567.2183 pm 294.4929
SASLA: 1359.6772 pm 250.1591
